# Rendering Time Series Plots to Video

This is a notebook for displaying a method of how to render a video out of a time series graph. 
The result is a graph building up from left to right, such that it looks like it is running through the video. 
The code is based on plotly, but should be easy to adapt to any other package

### Prerequisites
Do the below installs in the displayed order.

Have a virtual environment sourced beforehand!

openCV: https://www.pyimagesearch.com/opencv-tutorials-resources-guides/

In [ ]:
# necessary only once --> source environment
# !npm install -g electron@6.1.4 orca
# !pip install psutil requests cv2 natsort numpy pandas matplotlib plotly

### Notes
Printing out plotly images is done by the orca server, which is an electron app. Unfortunately it reacts sensibly to too much input, therefore there is a sleep timer in the end, before the f.write_image() - line. If running the notebook doesn't produce any output (neither in here, nor in the folder) it might be, that the orca server crashed. If that's the case, just shut it down (see end of notebook) and run the script again.

In [ ]:
import glob
import os
import time

import cv2
import natsort
import numpy as np
import pandas as pd
from sklearn.datasets import load_boston

from matplotlib.colors import to_rgba
import plotly.offline as py
import plotly.graph_objs as go
import plotly.io as pio
from plotly.tools import make_subplots

from IPython.display import clear_output

py.init_notebook_mode(connected=False)

In [ ]:
def to_rgba_str(name):
    rgba = [repr(val * (255 if i < 3 else 1)) for i, val in enumerate(to_rgba(name))]
    return "rgba(%s)" % ','.join(rgba)

In [ ]:
colors = [
    dict(color=to_rgba_str(name))
    for name in ["tab:blue", "tab:orange", 
                 "tab:green", "tab:red", 
                 "tab:purple", "tab:brown", 
                 "tab:pink", "tab:gray"]
]

other_colors = [
    dict(color=to_rgba_str(name))
    for name in ["#FFC512", "#CFAF7B", "#0485D1", "#FCE166", "#069AF3"]
]

In [ ]:
"""
Draw subsequent graphs and collect the images in a folder
"""

# get dataset
dataset = load_boston()
df = pd.DataFrame(dataset.data)

# full graph or hundreds of images for video rendering?
full_graph_only = False

font_family="sans-serif" # "sans-serif"; "PT Sans Narrow"; "Courier New"; "Raleway"

# set paths for printing out the images
single_image_path="/Users/niklas/Movies/DaVinci Resolve/iav-apx1/imgs/"
single_image_name="test.png"
collected_images_path="/Users/niklas/Movies/DaVinci Resolve/iav-apx1/imgs/test"

if not os.path.exists(collected_images_path) and full_graph_only==False:
    os.makedirs(collected_images_path)

# set window of x-values
xrange_start=0
xrange_end=180

# set window of y-values
yrange_start=0
yrange_end=25

# define figure widget
f = go.FigureWidget(make_subplots(
    rows=1,
    cols=1,
    shared_xaxes=True,
    shared_yaxes=True,
    vertical_spacing=0.1,
    print_grid=False,
    )
)

# modify axes
f.update_xaxes(range=[xrange_start, xrange_end],
              showticklabels=True)
f.update_yaxes(range=[yrange_start, yrange_end],
             showticklabels=True)

# modify layout
f.update_layout(
    autosize=False,
    width=1920,
    height=750,
    margin=go.layout.Margin(
        l=10,
        r=10,
        b=10,
        t=10,
        pad=1
    ),
    paper_bgcolor="white",  # "LightSteelBlue"
    plot_bgcolor="white",
    showlegend=True,
    legend=dict(
        x=0.84,
        y=0.15,
        traceorder="normal",
        font=dict(
            family=font_family,
            size=25,
            color="black"
        ),
        bgcolor="white",
        bordercolor="grey",
        borderwidth=0,
        orientation='v'
    )
)

f.layout.title = " "
f.layout.yaxis.title = " "  # "Pressure"
f.layout.yaxis.title.font.size = 25
f.layout.yaxis.title.font.family=font_family
f.layout.yaxis.tickfont.size = 17
f.layout.xaxis.tickfont.size = 17
f.layout.yaxis.gridcolor="lightgrey"
f.layout.xaxis.gridcolor="lightgrey"
f.layout.yaxis.showgrid=True
f.layout.xaxis.showgrid=True
f.layout.yaxis.zeroline=False
f.layout.xaxis.zeroline=False
f.layout.yaxis.showline=True
f.layout.xaxis.showline=True


# set amount of images used for rendering: read as "every nth image"
n = 1
df_adjusted = df[df.index % n == 0]

clear_output()

for index, row in df_adjusted.iterrows():
    f.data = []

    if index < xrange_start:
        time.sleep(0.001)
        continue
    elif index >= xrange_end:
        break

    if full_graph_only:
        df_res = df
    else:
        df_res = df.iloc[:index]

    f.add_scatter(y=df_res[2], marker=colors[2], name="Line 1")
    f.add_scatter(y=df_res[8], marker=colors[3], name="Line 7")

    if full_graph_only:
        f.write_image(os.path.join(single_image_path, single_image_name))
        break
    else:
        f.write_image(f"{collected_images_path}/fig{index}.png")
        # sleep to give orca server some time to breath, otherwise it will collapse
        time.sleep(0.05)

print("done")

In [ ]:
"""
Collect images from above script and render them into a video
"""
video_path="/Users/niklas/Movies/DaVinci Resolve/iav-apx1/vids"
video_name="test.mp4"

img_array = []
for filename in natsort.natsorted(glob.glob(f"{collected_images_path}/fig*.png")):
    img = cv2.imread(filename)
    img_array.append(img)
height, width, layers = img.shape
size = (width,height)

out = cv2.VideoWriter(os.path.join(video_path, video_name), cv2.VideoWriter_fourcc(*'mp4v'), 100, size) # DIVX, H264, MJPG

for i in range(len(img_array)):
    out.write(img_array[i])
out.release()
print("done")

In [ ]:
# check orca status, if needed
pio.orca.status

In [ ]:
# if orca server crashed, shut it down and run again script
pio.orca.shutdown_server()